In [1]:
import gc
import pandas as pd
import numpy as np

from speech_emotion.datasets import *
from speech_emotion.plots import *
from speech_emotion.features import *
from speech_emotion.modeling import *
from speech_emotion.evaluation import *

from keras.callbacks import EarlyStopping

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from IPython.display import clear_output as jupyter_clear_output

## EDA

In [2]:
def eda(data_df):
    plt.title('Count of Emotions', size=16)
    sns.countplot(x=data_df.Emotions)
    plt.ylabel('Count', size=12)
    plt.xlabel('Emotions', size=12)
    sns.despine(top=True, right=True, left=False, bottom=False)
    plt.show()

In [3]:
def plots(data_df):
    emotion = 'happy'
    path = np.array(data_df.Path[data_df.Emotions == emotion])[0]
    display_waveplot(path, emotion)
    display_spectrogram(path, emotion)
    display_melspectrogram(path, emotion)

## Train Model

In [8]:
def modeling(x_train, y_train, dataset, v=False):
    # model = build_model(shape=x_train[0].shape, classes=y.shape[1])
    model = create_model(x_train[0].shape, y_train.shape[1], v)

    es = EarlyStopping(
        monitor='loss', patience=2
    )

    history = model.fit(
        x_train, y_train,
        batch_size=500,
        epochs=40,
        verbose=1,
        callbacks=[es],
        # validation_split=0.2
    )
    
    del x_train, y_train
    gc.collect()

    model.save(f'{dataset}_model_4.h5')

    return model, history

## Main

In [9]:
if __name__ == "__main__":
    global DATASET, CLASSES, RESET

    RESET = False
    CLASSES = ["happy", "sad", "angry", "neutral"]

    for i in ["ALL"]:
        DATASET = i
        print(f"DATASET: {DATASET}\n")

        # Loading data
        data_df, x, y = load_x_y(RESET, CLASSES, DATASET)
        print(data_df.head())
        print("---------------------\n\n")

        print("Features:", x.shape)
        print("Labels:", y.shape)
        print("---------------------\n\n")

        # EDA
        eda(data_df)
        plots(data_df)

        del data_df
        gc.collect()

        jupyter_clear_output(wait=True)

        # Splitting data
        x_train, x_test, y_train, y_test = split(x, y)

        del x, y, x_test, y_test
        gc.collect()

        jupyter_clear_output(wait=True)

        # Modeling
        model, history = modeling(x_train, y_train, DATASET)

        display_model_history(history.history, val=False)

        jupyter_clear_output(wait=True)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.